In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
# import sys
# import os

# # Print debugging information
# print(f"Python path: {sys.executable}")
# print(f"Python version: {sys.version}")
# print(f"sys.path: {sys.path}")

# try:
#     from langchain_openai import ChatOpenAI
#     print("Successfully imported ChatOpenAI")
# except ImportError as e:
#     print(f"Import error: {e}")
#     print("Try running: pip install langchain-openai")
#     sys.exit(1)

# # Initialize the ChatOpenAI instance with your API key
# llm = ChatOpenAI(
#     api_key=os.environ.get("OPENAI_API_KEY")
# )

# # Test the chat
# response = llm.invoke("Hello, world!")
# print(response)

In [55]:
from langsmith import Client

client = Client()

In [58]:
# Convert Dataset to pandas DataFrame to allow item assignment
# df = df.to_pandas()
def make_dataset_csv():
    import pandas as pd

    # splits = {'dev': 'data/dev-00000-of-00001.parquet', 'test': 'data/test-00000-of-00001.parquet', 'train': 'data/train-00000-of-00001.parquet'}
    # df = pd.read_parquet("hf://datasets/princeton-nlp/SWE-bench/" + splits["dev"])
    from datasets import load_dataset

    ds = load_dataset("princeton-nlp/SWE-bench")
    df = ds['dev']
    # Convert to pandas DataFrame first
    import pandas as pd
    df = pd.DataFrame(df)
    df['version'] = df['version'].apply(lambda x: f"version:{x}")
    df.to_csv("SWE-bench-dev.csv",index=False)
    return df

def upload_dataset_programmatically(df):
    dataset = client.upload_csv(
        csv_file="SWE-bench-dev.csv",
        input_keys=list(df.columns),
        output_keys=[],
        name="swe-bench-programatic-upload",
        description="SWE-bench dataset",
        data_type="kv"
    )
upload_dataset_programmatically(make_dataset_csv())

In [59]:
from langsmith import evaluate


def predict(inputs: dict):
    return {"instance_id":inputs['instance_id'],"model_patch":"None","model_name_or_path":"test-model"}


In [60]:

result = evaluate(
    predict,
    data=client.list_examples(dataset_id="20bfdc4c-5515-4391-909c-9ba463a23817"),
)

View the evaluation results for experiment: 'best-uncle-91' at:
https://smith.langchain.com/o/22d9505f-5add-45d4-84e0-97f2aa494e0d/datasets/20bfdc4c-5515-4391-909c-9ba463a23817/compare?selectedSessions=cbe445ed-a2b8-4f18-bb2f-0dfa43fd6dfb




225it [00:01, 157.72it/s]


In [39]:
# import json

# with open('evaluation_results.json', 'w') as f:
#     json.dump(dict(result.to_pandas()), f, indent=2)

# dict(result.to_pandas())

In [40]:
help(type(result))

Help on class ExperimentResults in module langsmith.evaluation._runner:

class ExperimentResults(builtins.object)
 |  ExperimentResults(experiment_manager: '_ExperimentManager', blocking: 'bool' = True)
 |  
 |  Represents the results of an evaluate() call.
 |  
 |  This class provides an iterator interface to iterate over the experiment results
 |  as they become available. It also provides methods to access the experiment name,
 |  the number of results, and to wait for the results to be processed.
 |  
 |  Methods:
 |      experiment_name() -> str: Returns the name of the experiment.
 |      wait() -> None: Waits for the experiment data to be processed.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, experiment_manager: '_ExperimentManager', blocking: 'bool' = True)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __iter__(self) -> 'Iterator[ExperimentResultRow]'
 |  
 |  __len__(self) -> 'int'
 |  
 |  __repr__(self) -> 'str'
 |      Return rep

In [41]:
# import json

# with open('evaluation_results.json', 'w') as f:
#     json.dump(result.dict(), f, indent=2)


In [61]:
from swebench.harness.run_evaluation import run_instances
import resource
import docker
from swebench.harness.docker_utils import list_images, clean_images
from swebench.harness.docker_build import build_env_images
from pathlib import Path
import json
import os

RUN_EVALUATION_LOG_DIR = Path("logs/run_evaluation")
LANGSMITH_EVALUATION_DIR = './langsmith_feedback/feedback.json'

def convert_runs_to_langsmith_feedback(
        predictions: dict,
        full_dataset: list,
        run_id: str
    ) -> float:
    """
    Convert logs from docker containers into LangSmith feedback.

    Args:
        predictions (dict): Predictions dict generated by the model
        full_dataset (list): List of all instances
        run_id (str): Run ID
    """
    feedback_for_all_instances = {}

    for instance in full_dataset:
        feedback_for_instance = []
        instance_id = instance['instance_id']
        prediction = predictions[instance_id]
        if prediction.get("model_patch", None) in ["", None]:
            # Prediction returned an empty patch
            feedback_for_all_instances[prediction['run_id']] = [{"key":"non-empty-patch","score":0},
                                                                {"key":"completed-patch","score":0},
                                                                {"key":"resolved-patch","score":0}]
            continue
        feedback_for_instance.append({"key":"non-empty-patch","score":1})
        report_file = (
            RUN_EVALUATION_LOG_DIR
            / run_id
            / prediction["model_name_or_path"].replace("/", "__")
            / prediction['instance_id']
            / "report.json"
        )
        if report_file.exists():
            # If report file exists, then the instance has been run
            feedback_for_instance.append({"key":"completed-patch","score":1})
            report = json.loads(report_file.read_text())
            # Check if instance actually resolved the PR
            if report[instance_id]["resolved"]:
                feedback_for_instance.append({"key":"resolved-patch","score":1})
            else:
                feedback_for_instance.append({"key":"resolved-patch","score":0})
        else:
            # The instance did not run successfully
            feedback_for_instance += [{"key":"completed-patch","score":0},{"key":"resolved-patch","score":0}]
        feedback_for_all_instances[prediction['run_id']] = feedback_for_instance

    os.makedirs(os.path.dirname(LANGSMITH_EVALUATION_DIR), exist_ok=True)
    with open(LANGSMITH_EVALUATION_DIR, 'w') as json_file:
        json.dump(feedback_for_all_instances, json_file)

def evaluate_predictions(
        dataset: list,
        predictions: list,
        max_workers: int,
        force_rebuild: bool,
        cache_level: str,
        clean: bool,
        open_file_limit: int,
        run_id: str,
        timeout: int,
    ):
    """
    Run evaluation harness for the given dataset and predictions.
    """
    # set open file limit
    assert len(run_id) > 0, "Run ID must be provided"
    resource.setrlimit(resource.RLIMIT_NOFILE, (open_file_limit, open_file_limit))
    client = docker.from_env()

    existing_images = list_images(client)
    print(f"Running {len(dataset)} unevaluated instances...")

    try:
        # build environment images + run instances
        build_env_images(client, dataset, force_rebuild, max_workers)
        run_instances(predictions, dataset, cache_level, clean, force_rebuild, max_workers, run_id, timeout)
    except Exception as e:
        print(f"Docker image access error: {e}")
        return
    print("No docker image access error")
    # clean images + make final report
    clean_images(client, existing_images, cache_level, clean)
    # import pdb;pdb.set_trace()
    convert_runs_to_langsmith_feedback(predictions,dataset,run_id)
    # pdb.set_trace()
    pass
    pass
    

In [62]:
dataset = []
predictions = {}
for res in result:
    predictions[res['run'].outputs['instance_id']] = {**res['run'].outputs,**{"run_id":str(res['run'].id)}}
    dataset.append(res['run'].inputs['inputs'])
for d in dataset:
    d['version'] = d['version'].split(":")[1]

In [63]:
!docker login

Authenticating with existing credentials...
Login Succeeded


In [68]:
evaluate_predictions(dataset[:5],predictions,max_workers=8,force_rebuild=False,cache_level="env",clean=False \
                     ,open_file_limit=4096,run_id="test",timeout=1_800)

Running 5 unevaluated instances...
Base image sweb.base.py.arm64:latest already exists, skipping build.
Base images built successfully.
Total environment images to build: 2


  0%|          | 0/2 [00:00<?, ?it/s]Traceback (most recent call last):
  File "/Users/matthew.khoriaty/Desktop/research/coordination_among_ai/SWE-bench/swebench/harness/docker_build.py", line 155, in build_image
    raise docker.errors.BuildError(
docker.errors.BuildError: The command '/bin/sh -c /bin/bash -c "source ~/.bashrc && /root/setup_env.sh"' returned a non-zero code: 1

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/matthew.khoriaty/Desktop/research/coordination_among_ai/SWE-bench/swebench/harness/utils.py", line 92, in run_threadpool
    future.result()
  File "/Users/matthew.khoriaty/opt/anaconda3/envs/langchain/lib/python3.11/concurrent/futures/_base.py", line 449, in result
    return self.__get_result()
           ^^^^^^^^^^^^^^^^^^^
  File "/Users/matthew.khoriaty/opt/anaconda3/envs/langchain/lib/python3.11/concurrent/futures/_base.py", line 401, in __get_result
    raise self._exception
  File "/U

<class 'swebench.harness.docker_build.BuildImageError'>: Error building image sweb.env.py.arm64.f8f7816be1e30e5ea7547a:latest: The command '/bin/sh -c /bin/bash -c "source ~/.bashrc && /root/setup_env.sh"' returned a non-zero code: 1
Check (logs/build_images/env/sweb.env.py.arm64.f8f7816be1e30e5ea7547a__latest/build_image.log) for more information.
<class 'swebench.harness.docker_build.BuildImageError'>: Error building image sweb.env.py.arm64.63ba17657a55525251ae86:latest: The command '/bin/sh -c /bin/bash -c "source ~/.bashrc && /root/setup_env.sh"' returned a non-zero code: 1
Check (logs/build_images/env/sweb.env.py.arm64.63ba17657a55525251ae86__latest/build_image.log) for more information.
2 environment images failed to build.


Running 5 instances...


5 ran successfully, 0 failed: 100%|██████████| 5/5 [00:00<00:00,  7.00it/s]

Error building image pylint-dev__astroid-1333: 404 Client Error for http+docker://localhost/v1.47/images/create?tag=latest&fromImage=swebench%2Fsweb.eval.arm64.pylint-dev_1776_astroid-1333: Not Found ("pull access denied for swebench/sweb.eval.arm64.pylint-dev_1776_astroid-1333, repository does not exist or may require 'docker login': denied: requested access to the resource is denied")
Check (logs/run_evaluation/test/test-model/pylint-dev__astroid-1333/run_instance.log) for more information.
Error building image pvlib__pvlib-python-1623: 404 Client Error for http+docker://localhost/v1.47/images/create?tag=latest&fromImage=swebench%2Fsweb.eval.arm64.pvlib_1776_pvlib-python-1623: Not Found ("pull access denied for swebench/sweb.eval.arm64.pvlib_1776_pvlib-python-1623, repository does not exist or may require 'docker login': denied: requested access to the resource is denied")
Check (logs/run_evaluation/test/test-model/pvlib__pvlib-python-1623/run_instance.log) for more information.
Erro

Cleaning cached images...
Removed 0 images.
> /var/folders/g_/12fwrqfs2lgcf9rnql4m440dx849vl/T/ipykernel_46110/3502636115.py(97)evaluate_predictions()
     95     clean_images(client, existing_images, cache_level, clean)
     96     import pdb;pdb.set_trace()
---> 97     convert_runs_to_langsmith_feedback(predictions,dataset,run_id)
     98     pdb.set_trace()
     99     pass

> /var/folders/g_/12fwrqfs2lgcf9rnql4m440dx849vl/T/ipykernel_46110/3502636115.py(99)evaluate_predictions()
     97     convert_runs_to_langsmith_feedback(predictions,dataset,run_id)
     98     pdb.set_trace()
---> 99     pass
    100     pass
    101 

    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]



In [69]:
from langsmith import evaluate_existing
from langsmith.schemas import Example, Run
with open(LANGSMITH_EVALUATION_DIR, 'r') as json_file:
    langsmith_eval = json.load(json_file)
keyerrorcount = 0
def swe_bench_evaluator(run: Run, example: Example):

    return {"results": langsmith_eval.get(str(run.id), "No results found, not run for this one")}

experiment_name = result.experiment_name
evaluate_existing(experiment_name, evaluators=[swe_bench_evaluator])

View the evaluation results for experiment: 'best-uncle-91' at:
https://smith.langchain.com/o/22d9505f-5add-45d4-84e0-97f2aa494e0d/datasets/20bfdc4c-5515-4391-909c-9ba463a23817/compare?selectedSessions=cbe445ed-a2b8-4f18-bb2f-0dfa43fd6dfb




0it [00:00, ?it/s]2025-02-11 22:22:44,895 - langsmith.evaluation._runner - ERROR - Error running evaluator <DynamicRunEvaluator swe_bench_evaluator> on run 6c7690b6-7fb5-4cc3-8775-d790f1e4b1cb: ValueError("Expected an EvaluationResult object, or dict with a metric 'key' and optional 'score' or categorical 'value'; got N")
Traceback (most recent call last):
  File "/Users/matthew.khoriaty/opt/anaconda3/envs/langchain/lib/python3.11/site-packages/langsmith/evaluation/_runner.py", line 1634, in _run_evaluators
    evaluator_response = evaluator.evaluate_run(
                         ^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/matthew.khoriaty/opt/anaconda3/envs/langchain/lib/python3.11/site-packages/langsmith/evaluation/evaluator.py", line 336, in evaluate_run
    return self._format_result(result, source_run_id)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/matthew.khoriaty/opt/anaconda3/envs/langchain/lib/python3.11/site-packages/langsmith/evaluation/evaluator.py", lin

,inputs.repo,inputs.patch,inputs.version,inputs.created_at,inputs.hints_text,inputs.test_patch,inputs.base_commit,inputs.instance_id,inputs.FAIL_TO_PASS,inputs.PASS_TO_PASS,...,outputs.model_patch,outputs.model_name_or_path,error,feedback.swe_bench_evaluator,execution_time,example_id,id,feedback.non-empty-patch,feedback.completed-patch,feedback.resolved-patch
0,pylint-dev/astroid,diff --git a/astroid/nodes/node_ng.py b/astroi...,version:2.10,2022-01-14T21:14:48Z,"Actually, it seems a decorator is a red herrin...",diff --git a/tests/unittest_builder.py b/tests...,cfd9e74f7b4cbac08357cadec03c736501368afa,pylint-dev__astroid-1351,"[""tests/unittest_builder.py::FromToLineNoTest:...","[""tests/unittest_builder.py::FromToLineNoTest:...",...,None,test-model,None,NaN,0.000136,ffd119a9-29f8-436a-8984-9e1ec12f37ee,6c7690b6-7fb5-4cc3-8775-d790f1e4b1cb,NaN,NaN,NaN
1,pydicom/pydicom,diff --git a/pydicom/filewriter.py b/pydicom/f...,version:1.4,2020-04-17T01:12:48Z,,diff --git a/pydicom/tests/test_filereader.py ...,caf0db105ddf389ff5025937fd5f3aa1e61e85e7,pydicom__pydicom-1076,"[""pydicom/tests/test_filereader.py::TestReader...","[""pydicom/tests/test_filereader.py::TestReader...",...,None,test-model,None,NaN,0.000244,ff7bed2b-a486-40dc-a0c3-3f64878ed455,7f269e1e-1600-4a35-a37e-4259a23e92d3,NaN,NaN,NaN
2,pylint-dev/astroid,diff --git a/astroid/brain/brain_datetime.py b...,version:3.0,2023-06-22T03:00:19Z,,diff --git a/tests/brain/test_brain.py b/tests...,efb34f2b84c9f019ffceacef3448d8351563b6a2,pylint-dev__astroid-2219,"[""tests/brain/test_brain.py::CollectionsDequeT...",[],...,None,test-model,None,NaN,0.000328,ff382806-b03b-4316-9a92-d81ab6dd1a04,553d7768-b269-4c3a-b2e9-03cc794309d5,NaN,NaN,NaN
3,sqlfluff/sqlfluff,diff --git a/src/sqlfluff/rules/L039.py b/src/...,version:0.6,2021-10-22T18:23:33Z,Does running `sqlfluff fix` again correct the ...,diff --git a/test/rules/std_L003_L036_L039_com...,a1579a16b1d8913d9d7c7d12add374a290bcc78c,sqlfluff__sqlfluff-1733,"[""test/rules/std_L003_L036_L039_combo_test.py:...","[""test/rules/std_L016_L36_combo_test.py::test_...",...,None,test-model,None,NaN,0.000208,ff29f60a-5881-43e5-b9ce-b55eec2aba8e,638ffedf-f975-4ed0-97cd-71bbc0ae108e,NaN,NaN,NaN
4,pvlib/pvlib-python,diff --git a/pvlib/ivtools/sdm.py b/pvlib/ivto...,version:0.9,2023-04-27T14:29:30Z,,diff --git a/pvlib/tests/test_pvsystem.py b/pv...,30c62e368529df01faa609d6b38456a7b0db9b53,pvlib__pvlib-python-1719,"[""pvlib/tests/test_pvsystem.py::test_v_from_i[...","[""pvlib/tests/test_pvsystem.py::test_PVSystem_...",...,None,test-model,None,NaN,0.000202,fe5f9897-72ef-4db9-a5cd-92373ab5469c,d9269edb-d960-4117-b306-e5cced0884b8,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,pvlib/pvlib-python,diff --git a/pvlib/iam.py b/pvlib/iam.py\n--- ...,version:0.9,2023-03-24T10:46:42Z,,diff --git a/pvlib/tests/test_iam.py b/pvlib/t...,40e9e978c170bdde4eeee1547729417665dbc34c,pvlib__pvlib-python-1707,"[""pvlib/tests/test_iam.py::test_physical_n1_L0""]","[""pvlib/tests/test_iam.py::test_ashrae"", ""pvli...",...,None,test-model,None,NaN,0.000275,0299893e-0c12-4564-ba45-833f024c2c91,cdd8a49a-7dc3-4f00-a7ce-d91c61d93735,1.0,0.0,0.0
221,sqlfluff/sqlfluff,diff --git a/src/sqlfluff/rules/L027.py b/src/...,version:0.11,2022-04-04T20:29:42Z,This is tricky.\r\n\r\nBasicaly L026 works to ...,diff --git a/test/fixtures/rules/std_rule_case...,47c8bb29104761474e455ef2e6fdaa7a8cc20a56,sqlfluff__sqlfluff-2998,"[""test/rules/std_test.py::test__rules__std_fil...","[""test/rules/std_test.py::test__rules__std_fil...",...,None,test-model,None,NaN,0.000547,029542a0-ea97-466d-81c4-81852ac72de2,23ba5b76-442c-4a3c-aa80-de5dd5c4c6a2,1.0,0.0,0.0
222,pvlib/pvlib-python,diff --git a/docs/examples/adr-pvarray/plot_si...,version:0.8,2022-12-21T16:44:19Z,,diff --git a/pvlib/tests/iotools/test_tmy.py b...,30c62e368529df01faa609d6b38456a7b0db9b53,pvlib__pvlib-python-1623,"[""pvlib/tests/iotools/test_tmy.py::test_read_t...","[""pvlib/tests/ioto

In [70]:
# from langsmith import evaluate_existing
# from langsmith.schemas import Example, Run

# def swe_bench_evaluator(run: Run, example: Example):
#     with open(LANGSMITH_EVALUATION_DIR, 'r') as json_file:
#         langsmith_eval = json.load(json_file)
#     result = langsmith_eval.get(str(run.id))
#     if result is None:
#         # Log a warning or handle the missing result as needed.
#         #print(f"Warning: No evaluation result found for run id: {run.id}")
#         return {"results": None, "error": "No result found for this run id."}
#     return {"results": result}

# experiment_name = result.experiment_name
# evaluate_existing(experiment_name, evaluators=[swe_bench_evaluator])